<a href="https://colab.research.google.com/github/zhengyouqiang/Exercise_pytorchvideo/blob/main/X3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gradio pytorchvideo

     |████████████████████████████████| 2.0 MB 20.5 MB/s 
     |████████████████████████████████| 128 kB 50.8 MB/s 
     |████████████████████████████████| 206 kB 49.4 MB/s 
     |████████████████████████████████| 1.9 MB 39.9 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 37.2 MB 32 kB/s 
     |████████████████████████████████| 596 kB 42.3 MB/s 
     |████████████████████████████████| 3.5 MB 37.3 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
     |████████████████████████████████| 961 kB 41.8 MB/s 


In [ ]:
import torch
# Choose the `x3d_s` model 
import json
import urllib
import time
from pytorchvideo.data.encoded_video import EncodedVideo

from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample
)
import torch
# Choose the `slow_r50` model
from pytorchvideo.models.hub import (  # noqa: F401, E402
    c2d_r50,
    csn_r101,
    efficient_x3d_s,
    efficient_x3d_xs,
    i3d_r50,
    mvit_base_16,
    mvit_base_16x4,
    mvit_base_32x3,
    r2plus1d_r50,
    slow_r50,
    slow_r50_detection,
    slowfast_16x8_r101_50_50,
    slowfast_r50,
    slowfast_r50_detection,
    slowfast_r101,
    x3d_l,
    x3d_m,
    x3d_s,
    x3d_xs,
) 
model=x3d_s(pretrained=True)

import gradio as gr

#Video
torch.hub.download_url_to_file('https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4', 'archery.mp4')



# Set to GPU or CPU
device = "cuda"
model = model.eval()
model = model.to(device)

json_url = "https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json"
json_filename = "kinetics_classnames.json"
try: urllib.URLopener().retrieve(json_url, json_filename)
except: urllib.request.urlretrieve(json_url, json_filename)

with open(json_filename, "r") as f:
    kinetics_classnames = json.load(f)

# Create an id to label name mapping
kinetics_id_to_classname = {}
for k, v in kinetics_classnames.items():
    kinetics_id_to_classname[v] = str(k).replace('"', "")

mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
frames_per_second = 30
model_transform_params  = {
    "x3d_xs": {
        "side_size": 182,
        "crop_size": 182,
        "num_frames": 4,
        "sampling_rate": 12,
    },
    "x3d_s": {
        "side_size": 182,
        "crop_size": 182,
        "num_frames": 13,
        "sampling_rate": 6,
    },
    "x3d_m": {
        "side_size": 256,
        "crop_size": 256,
        "num_frames": 16,
        "sampling_rate": 5,
    }
}

# Get transform parameters based on model
transform_params = model_transform_params[model_name]

# Note that this transform is specific to the slow_R50 model.
transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(transform_params["num_frames"]),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=transform_params["side_size"]),
            CenterCropVideo(
                crop_size=(transform_params["crop_size"], transform_params["crop_size"])
            )
        ]
    ),
)

# The duration of the input clip is also specific to the model.
clip_duration = (transform_params["num_frames"] * transform_params["sampling_rate"])/frames_per_second

def x3dpred(video):

  # Select the duration of the clip to load by specifying the start and end duration
  # The start_sec should correspond to where the action occurs in the video
  start_sec = 0
  end_sec = start_sec + clip_duration

  # Initialize an EncodedVideo helper class and load the video
  video = EncodedVideo.from_path(video)

  # Load the desired clip
  video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

  # Apply a transform to normalize the video input
  video_data = transform(video_data)

  # Move the inputs to the desired device
  inputs = video_data["video"]
  inputs = inputs.to(device)


  # Pass the input clip through the model
  preds = model(inputs[None, ...])

  # calculate the time of prediction
  start=time.time()
  # Get the predicted classes
  post_act = torch.nn.Softmax(dim=1)
  preds = post_act(preds)
  pred_classes = preds.topk(k=5).indices[0]
  end=time.time()
  print(end-start)
  # Map the predicted classes to the label names
  pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes]
  return "%s" % ", ".join(pred_class_names)
  
inputs = gr.inputs.Video(label="Input Video")
outputs = gr.outputs.Textbox(label="Top 5 predicted labels")

title = "X3D"
description = "Gradio demo for X3D networks pretrained on the Kinetics 400 dataset. To use it, simply upload your video, or click one of the examples to load them. Read more at the links below."
article = "<p style='text-align: center'><a href='https://arxiv.org/abs/2004.04730'>X3D: Expanding Architectures for Efficient Video Recognition</a> | <a href='https://github.com/facebookresearch/pytorchvideo'>Github Repo</a></p>"

examples = [
    ['archery.mp4']
]

gr.Interface(x3dpred, inputs, outputs, title=title, description=description, article=article, examples=examples, analytics_enabled=False).launch(debug=True)